In [22]:
import pyaudio
import numpy as np
import librosa
from pythonosc import udp_client

# PyAudio configuration
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 44100
CHUNK = 2048

# OSC Configuration
OSC_IP = "127.0.0.1"
OSC_PORT = 6448
OSC_ADDRESS = "/wek/inputs"

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Initialize OSC client
client = udp_client.SimpleUDPClient(OSC_IP, OSC_PORT)

def frequency_to_note(frequency):
    # Mapping frequency to the nearest musical note (chromatically)
    if frequency is not None and frequency > 0:
        midi_num = librosa.hz_to_midi(frequency)  # Convert frequency to MIDI number
        note = librosa.midi_to_note(midi_num)  # Convert MIDI number to note
        return note
    return None



def process_audio(in_data, frame_count, time_info, status):
    audio_data = np.frombuffer(in_data, dtype=np.float32)

    # Calculate volume (RMS of the signal)
    volume = np.sqrt(np.mean(np.square(audio_data)))

    # Perform FFT and find the dominant frequency
    fft = np.abs(np.fft.fft(audio_data))
    freqs = np.fft.fftfreq(len(fft), 1.0/RATE)
    idx = np.argmax(fft)
    freq = freqs[idx]

    # Convert frequency to musical note
    note = frequency_to_note(freq)

    if note:
        # Ensure the volume is a float and note is a string
        volume = float(volume)
        note = str(note)

        # Print the values before sending (for debugging)
        print(f"Preparing to send Note: {note}, Volume: {volume}")

        try:
            # Send note and volume as an OSC message
            client.send_message(OSC_ADDRESS, [note, volume])
            print(f"Sent Note: {note}, Volume: {volume}")
        except ValueError as e:
            print(f"Error sending OSC message: {e}")

    return (in_data, pyaudio.paContinue)


# Open stream
stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK,
                    stream_callback=process_audio)

# Start the stream
stream.start_stream()

# Keep the stream open
try:
    while stream.is_active():
        pass
except KeyboardInterrupt:
    # Stop and close the stream and terminate PyAudio
    stream.stop_stream()
    stream.close()
    audio.terminate()
    print("Stream stopped")


Preparing to send Note: A3, Volume: 0.0026884160470217466
Sent Note: A3, Volume: 0.0026884160470217466
Preparing to send Note: A2, Volume: 0.0008479961543343961
Sent Note: A2, Volume: 0.0008479961543343961
Preparing to send Note: F1, Volume: 0.0023532293271273375
Sent Note: F1, Volume: 0.0023532293271273375
Preparing to send Note: F1, Volume: 0.005216529592871666
Sent Note: F1, Volume: 0.005216529592871666
Preparing to send Note: F1, Volume: 0.004399558529257774
Sent Note: F1, Volume: 0.004399558529257774
Preparing to send Note: F1, Volume: 0.004582252353429794
Sent Note: F1, Volume: 0.004582252353429794
Preparing to send Note: C2, Volume: 0.004519452806562185
Sent Note: C2, Volume: 0.004519452806562185
Preparing to send Note: F0, Volume: 0.004314101301133633
Sent Note: F0, Volume: 0.004314101301133633
Preparing to send Note: F1, Volume: 0.00435661431401968
Sent Note: F1, Volume: 0.00435661431401968
Preparing to send Note: A2, Volume: 0.003370176535099745
Sent Note: A2, Volume: 0.00337